In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2'
train_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])
test_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

In [3]:
train_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,34,0.016985,0.010509,0.000000,0.119705,0.000000,0.005568,0.000000,0.000000,0.449621,...,0.010347,0.006536,0.115255,0.000000,0.005462,0.000000,0.000000,0.351582,0.000000,0.142663
1854,Stany Zjednoczone,34,0.017544,0.011437,0.006623,0.125772,0.000000,0.006055,0.000000,0.000000,0.473915,...,0.011483,0.006536,0.116109,0.000000,0.006149,0.000000,0.000000,0.355254,0.000000,0.148196
1855,Stany Zjednoczone,34,0.018120,0.012690,0.006623,0.126702,0.000000,0.006817,0.000000,0.000000,0.478863,...,0.011840,0.013072,0.120435,0.000000,0.006760,0.000000,0.000000,0.368256,0.000000,0.150416
1856,Stany Zjednoczone,34,0.018712,0.013083,0.013245,0.131413,0.000000,0.007495,0.000000,0.000000,0.496390,...,0.011940,0.013072,0.122052,0.000000,0.007779,0.000000,0.000000,0.372171,0.000000,0.154770
1857,Stany Zjednoczone,34,0.019322,0.013194,0.013245,0.133173,0.000000,0.008625,0.000000,0.000000,0.501668,...,0.012561,0.013072,0.125706,0.000000,0.008162,0.000000,0.000000,0.382568,0.000000,0.156695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,35,0.005112,0.027868,0.013245,0.008810,0.002694,0.002540,0.003924,0.001725,0.046782,...,0.026320,0.013072,0.008216,0.002757,0.002288,0.004482,0.001754,0.036047,0.017440,0.021023
1997,Szwecja,35,0.005147,0.029056,0.013245,0.009214,0.003148,0.002537,0.004482,0.001754,0.048590,...,0.026822,0.013072,0.008650,0.002606,0.002525,0.004283,0.001892,0.035421,0.018473,0.020478
1998,Szwecja,35,0.005158,0.029609,0.013245,0.009687,0.002976,0.002800,0.004283,0.001892,0.047746,...,0.027630,0.013072,0.008228,0.002351,0.002198,0.004762,0.001918,0.035754,0.017144,0.020634


## Reshape the data

In [4]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_df, test_df)

## Build the model

In [5]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14600     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 34,851
Trainable params: 34,851
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [6]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3153/3153 [==============================] - 3s 739us/step - loss: 0.0031
Epoch 2/15
3153/3153 [==============================] - 2s 715us/step - loss: 0.0016
Epoch 3/15
3153/3153 [==============================] - 3s 915us/step - loss: 0.0013
Epoch 4/15
3153/3153 [==============================] - 2s 723us/step - loss: 0.0012
Epoch 5/15
3153/3153 [==============================] - 2s 765us/step - loss: 9.8404e-04
Epoch 6/15
3153/3153 [==============================] - 2s 698us/step - loss: 8.5116e-04
Epoch 7/15
3153/3153 [==============================] - 2s 681us/step - loss: 8.5182e-04
Epoch 8/15
3153/3153 [==============================] - 2s 689us/step - loss: 7.1177e-04
Epoch 9/15
3153/3153 [==============================] - 2s 742us/step - loss: 6.9789e-04
Epoch 10/15
3153/3153 [==============================] - 2s 684us/step - loss: 5.9225e-04
Epoch 11/15
3153/3153 [==============================] - 2s 682us/step - loss: 6.4587e-04
Epoch 12/15
3153/3153 [============

In [7]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [8]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_df.shape[1]-2)

26/26 [==============================] - 0s 440us/step


In [9]:
train_df_reset = train_df.reset_index()
test_df_reset = test_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_test.csv'))

## Charts

In [10]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

## Metrics

In [11]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_lstm)